# Mortgage Calculator with recurring early repayments

Based on [https://onladder.co.uk/blog/how-to-calculate-mortgage-repayments/](https://onladder.co.uk/blog/how-to-calculate-mortgage-repayments/)

Especially in the UK, you can make overpayments totalling up to 10\% of your original loan amount (not the current loan) during a 12 month period without penalty.

Boosting your monthly payments, even by rounding up, accelerates your mortgage repayment and reduces overall interest costs. For instance, by rounding it up, from £865 to £1,000. The excess goes into your equity, lowers the current principal, and therefore the cost for borrowing (due to interest).

This code demonstrates the impact of such overpayments on your mortgage.

## Parameters

All the numbers you need to get started

In [1]:
# Principal (starting balance) of the loan
mortgage_amount = 200000

# Annual interest rate (APRC)
# Can be expressed as either in percentage (e.g., 3.99%) or decimal (e.g., 0.0399)
interest_rate = 0.0399 

# Duration of your mortgage in years
mortgage_period = 30 

# Number of payments in total: if you make one mortgage payment every month for 25 years, that’s 25*12 = 300
# Duration of your mortgage in months
total_instalments = mortgage_period*12

# For displaying purposes
currency = "£"# "$" "€"

## Recurring payments
These are set of payments that recur montly with a start and end date.

In [2]:
# structured in periods. Each period has a starting time, end time, and total amount paid 
repayments = [{"start": 10,
               "end": 15,
               "amount_paid": 1000}, # giving £1,000 from the 10 to the 15 month (6 months)
              {"start": 16,
               "end": 22,
               "amount_paid": 1100}, # giving £1,100 from the 16 to the 22 month (8 months)
              {"start": 23,
               "end": 30,
               "amount_paid": 1200}] 

## Imports

In [3]:
import pandas as pd
from miscellaneous import *
from IPython.display import HTML, IFrame, display

## Original Summary
Without repayments

In [4]:
monthly_payment = payments(mortgage_amount,interest_rate,mortgage_period)
total_given = approx(monthly_payment*mortgage_period*12) 
print(f"Total amount borrowed {currency}{clean(mortgage_amount)}, with and interest rate of {clean(denormalise_interest_rate(interest_rate))}, and a repayment over {mortgage_period} ({mortgage_period*12} instalments)")
print(f"Montly payments set to {currency}{clean(monthly_payment)}")
print(f"Total payments {currency}{clean(total_given)}")
print(f"For each £1 borrowed you are will pay back {currency}{clean(total_given/mortgage_amount)}")

Total amount borrowed £200,000.00, with and interest rate of 3.99, and a repayment over 30 (360 instalments)
Montly payments set to £953.68
Total payments £343,324.80
For each £1 borrowed you are will pay back £1.72


In [5]:
# convert
check = check_recurring_repayments(repayments)
repayments_adj = convert_recurring_repayments(repayments)
print("Summary repayments:")
for k,v in repayments_adj.items():
    print(f"Month {k:3}, paid {currency}{v:4}")

Summary repayments:
Month  10, paid £1000
Month  11, paid £1000
Month  12, paid £1000
Month  13, paid £1000
Month  14, paid £1000
Month  15, paid £1000
Month  16, paid £1100
Month  17, paid £1100
Month  18, paid £1100
Month  19, paid £1100
Month  20, paid £1100
Month  21, paid £1100
Month  22, paid £1100
Month  23, paid £1200
Month  24, paid £1200
Month  25, paid £1200
Month  26, paid £1200
Month  27, paid £1200
Month  28, paid £1200
Month  29, paid £1200
Month  30, paid £1200


## Payment Schedule

In [6]:
columns=['Principal to date','Payment','Increased','Paid to date','Interest charged', 'Interest charged to date', 'Principal repaid', 'Principal repaid to date', 'Remaining principal']
table = pd.DataFrame(columns=columns, index=[x for x in range(1, total_instalments+1)])
remaining_principal = mortgage_amount
payment_to_date = 0
interest_paid_to_date = 0
principal_repaid_to_date = 0
to_break = False
for instalment in range(1, total_instalments+1):
    increased_payment = False
    this_month_payment = monthly_payment
    if instalment in repayments_adj:
        increased_payment = True
        this_month_payment = repayments_adj[instalment]
        if this_month_payment < monthly_payment:
            print(f"ERROR: For month {instalment} your repayment is set to {currency}{clean(this_month_payment)} instead of the original {currency}{clean(monthly_payment)}")
    
    principal_to_date = remaining_principal  
    curr_interest_paid = current_interest_paid(principal_to_date, interest_rate)
    if principal_to_date <= this_month_payment:
        this_month_payment = principal_to_date + curr_interest_paid
        to_break = True
    
    payment_to_date += this_month_payment
    
    interest_paid_to_date += curr_interest_paid
    principal_repaid = this_month_payment - curr_interest_paid
    principal_repaid_to_date += principal_repaid
    remaining_principal -= principal_repaid
    table.loc[instalment] = pd.Series({columns[0]:clean(principal_to_date),
                                         columns[1]:clean(this_month_payment),
                                         columns[2]:"Y" if increased_payment else "",
                                         columns[3]:clean(payment_to_date),
                                         columns[4]:clean(curr_interest_paid),
                                         columns[5]:clean(interest_paid_to_date),
                                         columns[6]:clean(principal_repaid),
                                         columns[7]:clean(principal_repaid_to_date),
                                         columns[8]:clean(remaining_principal)})
    if to_break:
        table = table.drop(range(instalment+1, total_instalments+1))
        break

### Details
- **Principal to date**:  This is the amount of the loan at a given time.

- **Payment**: This refers to the total amount of money paid toward the loan in a given period (usually monthly). This payment typically includes both principal and interest.

- **Paid to date**:  The total amount of money paid towards the loan since it originated. This includes both principal and interest portions of all payments made.

- **Increased**: Whether for this month there was an increased payment.

- **Interest charged**: The amount of interest that accrues on the loan for a specific period (e.g., a month). This is the cost of borrowing the money.

- **Interest charged to date**: The total amount of interest that has accrued on the loan since it originated.

- **Principal repaid**: The portion of a specific payment that goes towards reducing the original loan amount (the principal).

- **Principal repaid to date**: The total amount of the original loan amount that has been paid back since the loan originated.

- **Remaining principal**: The outstanding balance on the loan; this is the amount still owed. It's calculated as "Principal to date" minus "Principal repaid to date".

In [10]:
table.head(40)

,Principal to date,Payment,Increased,Paid to date,Interest charged,Interest charged to date,Principal repaid,Principal repaid to date,Remaining principal
1,"200,000.00",953.68,,953.68,665.00,665.00,288.68,288.68,"199,711.32"
2,"199,711.32",953.68,,"1,907.36",664.04,"1,329.04",289.64,578.32,"199,421.68"
3,"199,421.68",953.68,,"2,861.04",663.08,"1,992.12",290.60,868.92,"199,131.08"
4,"199,131.08",953.68,,"3,814.72",662.11,"2,654.23",291.57,"1,160.49","198,839.51"
5,"198,839.51",953.68,,"4,768.40",661.14,"3,315.37",292.54,"1,453.03","198,546.97"
6,"198,546.97",953.68,,"5,722.08",660.17,"3,975.54",293.51,"1,746.54","198,253.46"
7,"198,253.46",953.68,,"6,675.76",659.19,"4,634.73",294.49,"2,041.03","197,958.97"
8,"197,958.97",953.68,,"7,629.44",658.21,"5,292.94",295.47,"2,336.50","197,663.50"
9,"197,663.50",953.68,,"8,583.12",657.23,"5,950.18",296.45,"2,632.94","197,367.06"
10,"197,367.06","1,000.00",Y,"9,583.12",656.25,"6,606.42",343.75,"2,976.70","197,023.30"


## New Summary

In [8]:
print(f"Total amount borrowed {currency}{clean(mortgage_amount)}, with an initial interest rate of {clean(denormalise_interest_rate(interest_rate))}.")
print(f"Thanks to early repayments you shortened your mortgage to {instalment} instalments")
print(f"Montly payments was initially set to {currency}{clean(monthly_payment)}")
print(f"Then you performed {len(repayments_adj)} increased repayments, of a total of {currency}{clean(sum([rep-monthly_payment for _,rep in repayments_adj.items()]))}")
print(f"Your total repayment is {currency}{clean(payment_to_date)} (instead of the original {currency}{clean(total_given)})")
print(f"For each £1 borrowed you are will pay back {currency}{clean(payment_to_date/mortgage_amount)}")

Total amount borrowed £200,000.00, with an initial interest rate of 3.99.
Thanks to early repayments you shortened your mortgage to 350 instalments
Montly payments was initially set to £953.68
Then you performed 21 increased repayments, of a total of £3,272.72
Your total repayment is £336,731.46 (instead of the original £343,324.80)
For each £1 borrowed you are will pay back £1.68


## Export to Excel

In [9]:
output_file = "my_mortgage_analysis3.xlsx"
table.to_excel(output_file)